In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

import pymysql

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup
 
import re
import time 

import matplotlib.pyplot as plt
%matplotlib inline 

import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)

import seaborn as sns

In [ ]:
conn = pymysql.connect(host = "203.252.196.68",
                       user = 'db1614681', passwd = 'stat1234', db = 'sql1614681',
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

In [ ]:
query = "SELECT * FROM cpt_top100;"
cur.execute(query)
df_100=pd.DataFrame(cur.fetchall())[['ID','TIME_OF_CRAWL','TITLE','CUR_USER','PEAK_TODAY']]

query = "SELECT * FROM concur_price;"
cur.execute(query)
df1=pd.DataFrame(cur.fetchall())[['ID','TITLE','ORIGINAL_PRICE','FINAL_PRICE','DISCOUNT','LINK']]

query = "SELECT * FROM topsell_price;"
cur.execute(query)
df2=pd.DataFrame(cur.fetchall())[['ID','TITLE','ORIGINAL_PRICE','FINAL_PRICE','DISCOUNT','LINK']]

In [ ]:
conn.commit()
cur.close()
conn.close()

## 3개로 분류
1) longrun_game: 최고 매출 게임이면서 실시간 인기인 게임:   
2) topseller_game: 최고 매출 게임이지만 실시간 인기는 아닌 게임   
3) rising_game: 최고 매출 게임은 아니지만 실시간 인기인 게임

In [ ]:
# 최고매출&실시간
df_best=df2.loc[df2.TITLE.isin(list(df1.TITLE.unique())),]
df_best.reset_index(inplace=True)
del df_best['index']
# 최고매출
df_top=df2.loc[~df2.TITLE.isin(list(df1.TITLE.unique())),]
df_top = df_top.drop_duplicates(['TITLE'])
df_top.reset_index(inplace=True)
del df_top['index']
# 실시간
df_cur=df1.loc[~df1.TITLE.isin(list(df.TITLE.unique())),]
df_cur.reset_index(inplace=True)
del df_cur['index']
# 모든 게임
df_all=pd.concat([pd.concat([df_best,df_cur]),df_top])
df_all.reset_index(inplace=True)
del df_all['index']

In [ ]:
print(len(df_best))
print(len(df_top))
print(len(df_cur))
print(len(df_all))

In [ ]:
# 게임 이름, 발매일, 장르, 주요 검색 태그, 평가 반환하는 함수
def getDetails(bs_link):    
    getDetails_list=[] 
    title = bs_link.find('div', {'class':'apphub_AppName'}).text
    try:
        # 
        rel_date = bs_link.find('div',{'class':'date'}).text
        # 
        g_bl = bs_link.find('div',{'class':'block_content_inner'}).findAll('a')
        gen = ""
        for i in g_bl:
            if 'genre' in i['href']:
                gen = gen + ", " + i.text
        genre = gen[2:]
        #
        t_bl = bs_link.find('div',{'class':'glance_tags popular_tags'}).findAll('a')
        pop_tag = ""
        for i in t_bl:
            pop_tag = pop_tag + ", " + i.text.replace('\t','').replace('\n','')
            # pop_tag = pop_tag +  ", " + i['href'].split('/')[5] 
        popular_tag = pop_tag[2:]
        #
        try:
            rating = bs_link.find('div',{'class':'summary column'}).contents[1].text
        except:
            rating = 'NA'

    except:
        rel_date, genre, popular_tag, rating = 'NA','NA','NA','NA'
    
    dict_top = {'title': title,
                'genre': genre,
                'release_date': rel_date,
                'popular_tag': popular_tag,
                'rating': rating}

    getDetails_list.append(dict_top)
    
    return getDetails_list

## 1) longrun game
총 19개의 게임  

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")
options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Chrome/77.0.3865.90")

detail_list = []
for i in tqdm_notebook(range(0,len(df_best))):
    driver = webdriver.Chrome(executable_path = 'C:/Users/yumincho/chromedriver/chromedriver.exe',
                          options = options)
    url = "https://store.steampowered.com/"+df_best['LINK'][i]
    driver.get(url)
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'apphub_AppName')))
    except:
        driver.find_element_by_xpath('//*[@id="ageYear"]/option[98]').click()
        try:
            driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[3]/a[1]/span').click()
            req_link = driver.page_source
            bs = BeautifulSoup(req_link, 'html.parser') 
            driver.quit()
            detail_list.append(getDetails(bs))
        except:
            driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[4]/a[1]/span').click()
            req_link = driver.page_source
            bs = BeautifulSoup(req_link, 'html.parser') 
            driver.quit()
            detail_list.append(getDetails(bs))

In [ ]:
conn = pymysql.connect(host = "203.252.196.68",
                       user = 'db1614681', passwd = 'stat1234', db = 'sql1614681',
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
# 테이블 생성
myquery = """
CREATE TABLE longrun_game (
    ID BIGINT(7) NOT NULL AUTO_INCREMENT,
    TITLE VARCHAR(100), 
    RELEASE_DATE VARCHAR(100), 
    GENRE VARCHAR(100), 
    POPULAR_TAGS VARCHAR(1000),
    RATINGS VARCHAR(100),
    PRIMARY KEY(ID) ) 
    CHARSET=utf8mb4 ;
"""
cur.execute(myquery)
# DB 저장
for i in tqdm_notebook(range(0,len(detail_list))):
    query = """
            INSERT INTO longrun_game (TITLE, RELEASE_DATE, GENRE, POPULAR_TAGS, RATINGS)
            VALUES (%s, %s, %s, %s, %s)
            ;
        """
    cur.execute( query, (detail_list[i][0]['title'], 
                         detail_list[i][0]['release_date'], 
                         detail_list[i][0]['genre'], 
                         detail_list[i][0]['popular_tag'],
                         detail_list[i][0]['rating']))
conn.commit()
cur.close()
conn.close()

## 2) topseller game
총 99개의 게임(개별: 26개, 묶음: 13개(개별 66개))

In [ ]:
app_list = []
bundle_list = []
for i in range(0,len(df_top)):
    if df_top.LINK[i].find('app') == 1:
        app_dict = {'title':df_top.TITLE[i],
                    'link': df_top.LINK[i]}
        app_list.append(app_dict)   
    else:
        bundle_dict = {'title':df_top.TITLE[i],
                       'link':df_top.LINK[i]}
        bundle_list.append(bundle_dict)

### 26개 개별 게임

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")
options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Chrome/77.0.3865.90")

detail_list = []
for i in tqdm_notebook(range(0,len(app_list))):
    driver = webdriver.Chrome(executable_path = 'C:/Users/yumin/chromedriver/chromedriver.exe',
                          options = options)
    url = "https://store.steampowered.com/"+app_list[i]['link']
    driver.get(url)
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'apphub_AppName')))
    except:
        driver.find_element_by_xpath('//*[@id="ageYear"]/option[98]').click()
        try:
            driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[3]/a[1]/span').click()
            time.sleep(3)
            req_link = driver.page_source
            bs = BeautifulSoup(req_link, 'html.parser') 
            driver.quit()
            detail_list.append(getDetails(bs))
        except:
            driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[4]/a[1]/span').click()
            time.sleep(3)
            req_link = driver.page_source
            bs = BeautifulSoup(req_link, 'html.parser') 
            driver.quit()
            detail_list.append(getDetails(bs))

### 13개 번들 게임의 세부 링크 수집

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")
options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Chrome/77.0.3865.90")


link_list=[]
link_list_not = []

for i in tqdm_notebook(range(0,len(bundle_list))):
    link=[]
    driver = webdriver.Chrome(executable_path = 'C:/Users/yumin/chromedriver/chromedriver.exe',
                          options = options)
    url = "https://store.steampowered.com/"+bundle_list[i]['link']
    driver.get(url)

    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'package_totals_row')))
        req_link = driver.page_source
        bs = BeautifulSoup(req_link, 'html.parser') 
        driver.quit()
        a_tag=bs.findAll('a', {'class': 'tab_item_overlay'})
        for i in range(0,len(a_tag)):
            link.append(a_tag[i]['href'].split('com')[1])
    except:
        driver.find_element_by_xpath('//*[@id="ageYear"]/option[98]').click()
        try:
            driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[4]/a[1]/span').click()
            time.sleep(3)
            req_link = driver.page_source
            bs = BeautifulSoup(req_link, 'html.parser') 
            driver.quit()
            a_tag=bs.findAll('a', {'class': 'tab_item_overlay'})
            for i in range(0,len(a_tag)):
                link.append(a_tag[i]['href'].split('com')[1])
        except:
            driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[3]/a[1]/span').click()
            time.sleep(3)
            req_link = driver.page_source
            bs = BeautifulSoup(req_link, 'html.parser') 
            driver.quit()
            a_tag=bs.findAll('a', {'class': 'tab_item_overlay'})
            for i in range(0,len(a_tag)):
                link.append(a_tag[i]['href'].split('com')[1])
                
    if len(df_all.loc[df_all.LINK.isin(link),])==0:
        link_list.extend(link)
    else:
        link_list_not.extend(link)

In [ ]:
print(len(link_list_not))
print(len(link_list))

### 66개 게임

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")
options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Chrome/77.0.3865.90")

detail_list = []
for i in tqdm_notebook(range(0,ㅣlen(link_list))):
    driver = webdriver.Chrome(executable_path = 'C:/Users/yumin/chromedriver/chromedriver.exe',
                          options = options)
    url = "https://store.steampowered.com/"+link_list[i]
    driver.get(url)
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'apphub_AppName')))
        req_link = driver.page_source
        bs = BeautifulSoup(req_link, 'html.parser') 
        driver.quit()
        detail_list.append(getDetails(bs))
    except:
        driver.find_element_by_xpath('//*[@id="ageYear"]/option[98]').click()
        try:
            driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[3]/a[1]/span').click()
            time.sleep(3)
            req_link = driver.page_source
            bs = BeautifulSoup(req_link, 'html.parser') 
            driver.quit()
            detail_list.append(getDetails(bs))
        except:
            driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[4]/a[1]/span').click()
            time.sleep(3)
            req_link = driver.page_source
            bs = BeautifulSoup(req_link, 'html.parser') 
            driver.quit()
            detail_list.append(getDetails(bs))

In [ ]:
conn = pymysql.connect(host = "203.252.196.68",
                       user = 'db1614681', passwd = 'stat1234', db = 'sql1614681',
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
# 테이블 생성
myquery = """
CREATE TABLE topseller_game (
    ID BIGINT(7) NOT NULL AUTO_INCREMENT,
    TITLE VARCHAR(100), 
    RELEASE_DATE VARCHAR(100), 
    GENRE VARCHAR(100), 
    POPULAR_TAGS VARCHAR(1000),
    RATINGS VARCHAR(100),
    PRIMARY KEY(ID) ) 
    CHARSET=utf8mb4 ;
"""
cur.execute(myquery)
# DB 저장
for i in tqdm_notebook(range(0,len(detail_list))):
    query = """
            INSERT INTO topseller_game (TITLE, RELEASE_DATE, GENRE, POPULAR_TAGS, RATINGS)
            VALUES (%s, %s, %s, %s, %s)
            ;
        """
    cur.execute( query, (detail_list[i][0]['title'], 
                         detail_list[i][0]['release_date'], 
                         detail_list[i][0]['genre'], 
                         detail_list[i][0]['popular_tag'],
                         detail_list[i][0]['rating']))
conn.commit()
cur.close()
conn.close()

## 3) rising game
총 41개의 게임    

In [ ]:
detail_list = []
for i in tqdm_notebook(range(0,len(df_cur))):
    driver = webdriver.Chrome(executable_path = 'C:/Users/yumincho/chromedriver/chromedriver.exe',
                          options = options)
    url = "https://store.steampowered.com/"+df_cur['LINK'][i]
    driver.get(url)
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'apphub_AppName')))
        req_link = driver.page_source
        bs = BeautifulSoup(req_link, 'html.parser') 
        driver.quit()
        detail_list.append(getDetails(bs))
    except:
        driver.find_element_by_xpath('//*[@id="ageYear"]/option[98]').click()
        driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[3]/a[1]/span').click()
        time.sleep(3)
        req_link = driver.page_source
        bs = BeautifulSoup(req_link, 'html.parser') 
        driver.quit()
        detail_list.append(getDetails(bs))

In [ ]:
conn = pymysql.connect(host = "203.252.196.68",
                       user = 'db1614681', passwd = 'stat1234', db = 'sql1614681',
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
# 테이블 생성
myquery = """
CREATE TABLE rising_game (
    ID BIGINT(7) NOT NULL AUTO_INCREMENT,
    TITLE VARCHAR(100), 
    RELEASE_DATE VARCHAR(100), 
    GENRE VARCHAR(100), 
    POPULAR_TAGS VARCHAR(1000),
    RATINGS VARCHAR(100),
    PRIMARY KEY(ID) ) 
    CHARSET=utf8mb4 ;
"""
cur.execute(myquery)
# DB 저장
for i in tqdm_notebook(range(0,len(detail_list))):
    query = """
            INSERT INTO rising_game (TITLE, RELEASE_DATE, GENRE, POPULAR_TAGS, RATINGS)
            VALUES (%s, %s, %s, %s, %s)
            ;
        """
    cur.execute( query, (detail_list[i][0]['title'], 
                         detail_list[i][0]['release_date'], 
                         detail_list[i][0]['genre'], 
                         detail_list[i][0]['popular_tag'],
                         detail_list[i][0]['rating']))
conn.commit()
cur.close()
conn.close()